# Torch Dataset Utilities

"torchdatasetutils" produces torch [DataLoader](https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader) classes and utility functions for several imaging datasets.  This currently includes sets of images and annotations from [CVAT](https://github.com/openvinotoolkit/cvat), [COCO dataset](https://cocodataset.org/).  "torchdatasetutil" uses an s3 object storage to hold dataset data.  This enables training and test to be performed on nodes different from where the dataset is stored with application defined credentials.  It uses torch PyTorch worker threads to prefetch data for efficient GPU or CPU training and inference.

"torchdatasetutils" takes as an input the [pymlutil](https://pypi.org/project/pymlutil/).s3 object to access the object storage.

Two json or yaml dictionaries are loaded from the object storage to identify and process the dataset: the dataset description and class dictionary.  The the dataset description is unique for each type of dataset.  The class dictionary is common to all datasets and describes data transformation and data augmentation.

## Library structure
- pymlutil.s3: access to object torage
- [torchdatasetutil](https://pypi.org/project/torchdatasetutil/)
    - [gitcoco.getcoco](https://github.com/bhlarson/torchdatasetutil/blob/main/torchdatasetutil/getcoco.py#L25): function to load the [COCO dataset](https://cocodataset.org/) from internet archives into object storage
    - [cocostore](https://github.com/bhlarson/torchdatasetutil/blob/main/torchdatasetutil/cocostore.py)
        - [CocoStore](https://github.com/bhlarson/torchdatasetutil/blob/main/torchdatasetutil/cocostore.py#L17): class providing a python iterator over the coco dataset in object storage
        - [CocoDataset](https://github.com/bhlarson/torchdatasetutil/blob/main/torchdatasetutil/cocostore.py)" class implementing the pytorch [Dataset class](https://pytorch.org/docs/stable/data.html#dataset-types) for the CocoStore iterator
        - CreateCocoLoaders: function returning set of torch dataloders of CocoDatasets.
    - [imstore](https://github.com/bhlarson/torchdatasetutil/blob/main/torchdatasetutil/imstore.py)
        - ImagesStore: class providing python iterator over sets of images for dataset inference.
        - ImagesDataset: Torch dataset derived class augmenting and extaracting random crops for training and test.
        - CreateImageLoaders: function returning set of torch dataloaders of ImagesDatasets.


In [9]:
from pymlutil.s3 import s3store, Connect
from torchdatasetutil.imstore import ImagesStore, CreateImageLoaders
from torchdatasetutil.cocostore import CocoStore, CreateCocoLoaders

In [10]:
credentials = "creds.yaml"
s3, creds, s3def = Connect(credentials)
s3def

{'name': 'store',
 'type': 'trainer',
 'address': '198.211.145.1:30990',
 'tls': False,
 'sets': {'dataset': {'bucket': 'mllib',
   'prefix': 'data',
   'dataset_filter': ''},
  'trainingset': {'bucket': 'mllib',
   'prefix': 'training',
   'dataset_filter': ''},
  'model': {'bucket': 'mllib', 'prefix': 'model', 'dataset_filter': ''},
  'test': {'bucket': 'mllib', 'prefix': 'test', 'dataset_filter': ''}}}

In [ ]:
## Create images dataset
images_path = 'testimages'
dataset_name = 'testimages'
s3.PuthDir(s3def['sets']['dataset']['bucket'], images_path, dataset_name)

### imstore dataset
torchdatasetutil.imstore ImagesStore produces an [iterator object](https://docs.python.org/3/c-api/iterator.html) from a set of images in an object store.  

In [11]:
dataset_desc = 'annotations/lit/dataset.yaml'
dataset_desc_example = s3.GetDict(s3def['sets']['dataset']['bucket'],dataset_desc)
dataset_desc_example

{'bucket': 'mllib',
 'prefix': 'annotations/lit',
 'image_path': None,
 'image_pattern': '*.tif',
 'image_colorspace': 'grayscale',
 'label_path': None,
 'label_in_image_path': True,
 'label_pattern': '*_cls.png',
 'recursive': True}

### Class Dictionary

In [12]:
class_dictionary = 'model/crisplit/lit.json'
class_dictionary_example = s3.GetDict(s3def['sets']['dataset']['bucket'],class_dictionary)
class_dictionary_example

{'background': 0,
 'ignore': 255,
 'classes': 2,
 'objects': [{'id': 0,
   'name': 'unlabeled',
   'trainId': 0,
   'category': 'void',
   'display': False,
   'color': [0, 0, 0]},
  {'id': 1,
   'name': 'FibLine',
   'trainId': 1,
   'category': 'vehicle',
   'display': True,
   'color': [255, 0, 0]},
  {'id': 9,
   'name': 'FibLine',
   'trainId': 1,
   'category': 'vehicle',
   'display': True,
   'color': [255, 0, 0]}]}

In [13]:
bucket = s3def['sets']['dataset']['bucket']


dataset_desc = s3.GetDict(s3def['sets']['dataset']['bucket'],dataset_desc)


self.store = ImagesStore(s3, bucket, dataset_desc, class_dictionary)


class_dictionary = s3.GetDict(s3def['sets']['dataset']['bucket'],args.class_dict) 
os.makedirs(args.test_path, exist_ok=True)

store = CocoStore(s3, bucket=s3def['sets']['dataset']['bucket'], 
                    dataset_desc=args.dataset_train, 
                    image_paths=args.train_image_path, 
                    class_dictionary=args.class_dict, 
                    imflags=args.imflags)

for i, iman in enumerate(store):
    img = store.MergeIman(iman['img'], iman['ann'])
    write_path = '{}cocostoreiterator{:03d}.png'.format(args.test_path, i)
    cv2.imwrite(write_path,img)
    if i >= args.num_images:
        print ('test_iterator complete')
                break

IndentationError: unexpected indent (2819687617.py, line 25)